Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
See https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py

based on https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
or http://archive.is/3qsFl

Data download
English to French sentence pairs.
http://www.manythings.org/anki/fra-eng.zip
Lots of neat sentence pairs datasets can be found at:
http://www.manythings.org/anki/

In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

/home/seung/.venv/py3Keras/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/home/seung/.venv/py3Keras/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seung/.venv/py3Keras/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = './data/fra.txt'

In [3]:
!head $data_path

Go.	Va !
Run!	Cours !
Run!	Courez !
Fire!	Au feu !
Help!	À l'aide !
Jump.	Saute.
Stop!	Ça suffit !
Stop!	Stop !
Stop!	Arrête-toi !
Wait!	Attends !


In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text = line.split('\t')
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = '\t' + target_text + '\n'
        input_texts.append(input_text)
        target_texts.append(target_text)
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [5]:
print(input_characters)

[' ', '!', '$', '%', '&', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [6]:
print(target_characters)

['\t', '\n', ' ', '!', '$', '%', '&', "'", '(', ')', ',', '-', '.', '0', '1', '3', '5', '6', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'à', 'â', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '\u2009', '’', '\u202f']


In [7]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [8]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
        
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [9]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(10000, 16, 71)
(10000, 59, 94)
(10000, 59, 94)


num_encoder_tokens / num_decoder_tokens 이므로 one-hot

In [10]:
print(decoder_input_data[0,1:4])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [11]:
print(decoder_target_data[0,:3])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [12]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [13]:
print(state_h)

Tensor("lstm_1/while/Exit_2:0", shape=(?, 256), dtype=float32)


In [14]:
print(state_c)

Tensor("lstm_1/while/Exit_3:0", shape=(?, 256), dtype=float32)


Q. encoder_inputs 의 shape이 (None, num_encoder_tokens) 인데, 일단 일종의 차원만 지정해주나?

In [15]:
print(encoder_states)

[<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]


In [16]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

Check for the shape of decoder_inputs

In [17]:
print(decoder_inputs)

Tensor("input_2:0", shape=(?, ?, 94), dtype=float32)


In [18]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [19]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.9195 - val_loss: 0.9683
Epoch 2/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.7246 - val_loss: 0.7748
Epoch 3/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.6184 - val_loss: 0.7042
Epoch 4/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.5612 - val_loss: 0.6570
Epoch 5/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.5218 - val_loss: 0.6169
Epoch 6/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.4893 - val_loss: 0.5851
Epoch 7/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.4625 - val_loss: 0.5680
Epoch 8/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.4408 - val_loss: 0.5512
Epoch 9/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.4204 - val_loss: 0.5396
Epoch 10/100
800

8000/8000 [==============================] - 11s 1ms/step - loss: 0.0816 - val_loss: 0.6799
Epoch 79/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0804 - val_loss: 0.6828
Epoch 80/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0791 - val_loss: 0.6848
Epoch 81/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0775 - val_loss: 0.6881
Epoch 82/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0767 - val_loss: 0.6940
Epoch 83/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0756 - val_loss: 0.7025
Epoch 84/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0749 - val_loss: 0.7022
Epoch 85/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0737 - val_loss: 0.7094
Epoch 86/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0725 - val_loss: 0.7142
Epoch 87/100
8000/8000 [==============================] - 11s 1ms/st

In [47]:
# Save model
# See https://jovianlin.io/saving-loading-keras-models/
model.save('s2s_model.h5')

/home/seung/.venv/py3Keras/lib/python3.5/site-packages/keras/engine/topology.py:2344: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [45]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 71)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 94)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 335872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  359424      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [46]:
!ls

data  s2s_model_architecture.json  testseq2seq.ipynb


## Next: inference mode (sampling)

* Here's the drill:
* 1) encode input and retrieve initial decoder state
* 2) run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token
* 3) Repeat with the current target token and current states

In [29]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

In [24]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

decoder는 states inputs 만 있음

In [30]:
print(input_token_index.items())

dict_items([('c', 47), ('!', 1), ('%', 3), ('?', 20), ('q', 61), ('y', 69), ('6', 15), ('U', 41), ('e', 49), ('f', 50), ('J', 30), ('S', 39), ('4', 13), ('A', 21), ('a', 45), ('n', 58), ('b', 46), ('Q', 37), ('1', 10), ('K', 31), ('9', 18), ('C', 23), ('j', 54), ('m', 57), ('g', 51), ('H', 28), ('0', 9), ('B', 22), ('E', 25), ('u', 65), ('N', 34), (',', 6), ('2', 11), ('&', 4), ('7', 16), ('k', 55), ('i', 53), ('W', 43), ('M', 33), ('T', 40), ('F', 26), ('V', 42), ('w', 67), ('D', 24), ('$', 2), ('o', 59), ('Y', 44), ('z', 70), ('l', 56), (':', 19), ('r', 62), ('L', 32), ('p', 60), ('8', 17), ('s', 63), ('-', 7), ('x', 68), ('v', 66), ('t', 64), ('d', 48), ('3', 12), ('O', 35), ('5', 14), (' ', 0), ('G', 27), ('I', 29), ('.', 8), ('R', 38), ("'", 5), ('P', 36), ('h', 52)])


In [31]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [66]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        #print([target_seq] + states_value)
        # Sample a token
        #print(np.shape(output_tokens)) # (1, 1, 94)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        if (sampled_char != '\n'):
            print('{} <--- sampled_char : {}'.format(decoded_sentence, sampled_char))
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [67]:
input_seq0 =  encoder_input_data[0: 0+1]
print([input_seq0])
print(np.shape(input_seq0))

[array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)]
(1, 16, 71)


In [68]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

(1, 1, 94)
V <--- sampled_char : V
(1, 1, 94)
Va <--- sampled_char : a
(1, 1, 94)
Va  <--- sampled_char :  
(1, 1, 94)
Va ! <--- sampled_char : !
(1, 1, 94)
-
Input sentence: Go.
Decoded sentence: Va !

(1, 1, 94)
C <--- sampled_char : C
(1, 1, 94)
Co <--- sampled_char : o
(1, 1, 94)
Cou <--- sampled_char : u
(1, 1, 94)
Cour <--- sampled_char : r
(1, 1, 94)
Cours <--- sampled_char : s
(1, 1, 94)
Cours  <--- sampled_char :  
(1, 1, 94)
Cours ! <--- sampled_char : !
(1, 1, 94)
-
Input sentence: Run!
Decoded sentence: Cours !

(1, 1, 94)
C <--- sampled_char : C
(1, 1, 94)
Co <--- sampled_char : o
(1, 1, 94)
Cou <--- sampled_char : u
(1, 1, 94)
Cour <--- sampled_char : r
(1, 1, 94)
Cours <--- sampled_char : s
(1, 1, 94)
Cours  <--- sampled_char :  
(1, 1, 94)
Cours ! <--- sampled_char : !
(1, 1, 94)
-
Input sentence: Run!
Decoded sentence: Cours !

(1, 1, 94)
A <--- sampled_char : A
(1, 1, 94)
Au <--- sampled_char : u
(1, 1, 94)
Au  <--- sampled_char :  
(1, 1, 94)
Au f <--- sampled_char :

### TODO 
* load the saved model and decode sentence from the model (using the sampling..)